# 1 - Imports and defining functions

In [ ]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

## 2- Loading Data

In [ ]:
mesh1 = TriMesh('data/cat-00.off')
mesh2 = TriMesh('data/lion-00.off')
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

# 3 - Computing the functional map

**Computing descriptors**

In [ ]:
k1,k2 = 60,60 # Number of eigenvalues on source and Target
landmarks = np.loadtxt('data/landmarks.txt',dtype=int)[:5] # loading 5 landmarks
subsample_step = 5 # In order not to use too many descriptors*
descr_type = 'WKS' # WKS or HKS

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(n_ev=(k1,k2),subsample_step = subsample_step, landmarks = landmarks, descr_type=descr_type,verbose=True);

**Fitting the model**

In [ ]:
descr_mu = 1e0
lap_mu = 1e-3
descr_comm_mu=1e-1

model.fit(descr_mu = descr_mu,lap_mu = lap_mu, descr_comm_mu=descr_comm_mu, verbose=True)

**Visualizing the associated point to point map**

In [ ]:
p2p = model.p2p
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 4 - Refining the Functional Map
**ICP**

In [ ]:
model.icp_refine()
p2p = model.p2p
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p]
double_plot(mesh1,mesh2,cmap1,cmap2)

**Zoomout**

In [ ]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one

model.zoomout_refine(nit=15)
p2p = model.p2p
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p]
double_plot(mesh1,mesh2,cmap1,cmap2)